<a href="https://colab.research.google.com/github/vinhhiep96/06-House_Prices_Advanced_Regression-Techniques-Sergei_Neviadomski/blob/main/23_11_California_Housing_Prices.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# import library and load data



In [38]:
#import library
import pandas as pd
import numpy as np
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.graph_objects as go
import warnings
warnings.filterwarnings('ignore')

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

from sklearn.svm import SVR
from xgboost import XGBRegressor
from sklearn.linear_model import Ridge
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import BayesianRidge
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

#load data
from google.colab import drive
drive.mount('/content/drive')
path1='/content/drive/MyDrive/Colab Notebooks/jupyter/23.11 California Housing Prices/train.csv'
df1=pd.read_csv(path1)

#fillna
df1['LotFrontage']=df1['LotFrontage'].fillna(df1['LotFrontage'].mean())
df1['GarageYrBlt']=df1['GarageYrBlt'].fillna(df1['GarageYrBlt'].mean())
df1['MasVnrArea']=df1['MasVnrArea'].fillna(df1['MasVnrArea'].mean())
df1=df1.fillna('None')

df1.head(3)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,None,Reg,Lvl,AllPub,...,0,None,None,None,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,None,Reg,Lvl,AllPub,...,0,None,None,None,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,None,IR1,Lvl,AllPub,...,0,None,None,None,0,9,2008,WD,Normal,223500


# scale values

In [39]:
df2=df1.drop(columns='SalePrice') #train data
list1=[col for col in df2.columns if df2[col].dtype in['float64','int64']] #numeric values
list2=[col for col in df2.columns if df2[col].dtype not in['float64','int64']] #category values

df2[list1] = MinMaxScaler().fit_transform(df2[list1]) #scale numeric values
oe = OneHotEncoder(sparse=False, handle_unknown='ignore').fit(df2[list2]) #category numeric values
list2a = list(oe.get_feature_names_out(list2))
df2[list2a] = oe.transform(df2[list2])

df2.head(3)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,0.000000,0.235294,RL,0.150685,0.033420,Pave,None,Reg,Lvl,AllPub,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.000685,0.000000,RL,0.202055,0.038795,Pave,None,Reg,Lvl,AllPub,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.001371,0.235294,RL,0.160959,0.046507,Pave,None,IR1,Lvl,AllPub,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


In [40]:
# score models

In [41]:
# dict of models
models = {
    'SVR':SVR(),
    'XGBRegressor':XGBRegressor(),
    'Ridge':Ridge(),
    'ElasticNet':ElasticNet(),
    'SGDRegressor':SGDRegressor(),
    'BayesianRidge':BayesianRidge(),
    'LinearRegression':LinearRegression(),
    'RandomForestRegressor':RandomForestRegressor()
}
#split data
X_train,X_test,y_train,y_test = train_test_split(df2[list1 + list2a],
                                                 df3,
                                                 test_size=0.25,
                                                 random_state=42)
#score models
list3=[]
list4=[]
for x, y in models.items():
   A=y.fit(X_train, y_train).predict(X_test) #predict
   B=np.sqrt(mean_squared_error(y_test, A)) #score
   list3.append(x) #model name
   list4.append(B) #model score

df4=pd.DataFrame([list3, list4]).T.rename(columns=({0:'Model', 1:'Score'})).sort_values('Score')
df4.head(3)

,Model,Score
1,XGBRegressor,26985.234409
7,RandomForestRegressor,27147.560875
2,Ridge,29017.753785


# differential between predicted values and test values

In [43]:
df5=pd.DataFrame(y_test)
df6=pd.DataFrame()
fig=go.Figure()
for x, y in models.items():
   A=y.fit(X_train, y_train).predict(X_test) #predict values
   df5[x]=A
   df6[x]=abs(df5[x]-df5['SalePrice']) #differentials
   df7=df6.head(10).reset_index()  #10 differentials
   fig.add_trace(go.Bar(x=df7.index, y=df6[x], name=x))
df7=df6.head(10).reset_index()
fig.update_xaxes(dtick=1)
fig.show()